# PLAN NA TESTING METRYKI

- z macierzy korelacji wybrać lambdy ktore wyszly
- zrobić narzedzie które liczy score
- wygenerować graf - recznie(?) wygenerowac jakąś wizulizacje jego, jedna lepsza, jedna wyraźnie gorsza i popatrzeć czy metryka zwraca wynik który ma sens 
- w komponentach dac tez wersje komponentow bez zmian z artykulu
- porównać wyniki na 3 kategoriach grafu (BA ER SBM) i pomysleć czy przy wybieraniu lambd brać pod uwagę rodzaj tego grafu

In [1]:
import sys
from pathlib import Path

# sys.path.insert(0,str(Path(r'C:\Users\ku51015\CHMURA\mystuff\Graph-Vizualisation-Rating-Metric').resolve()))
sys.path.insert(0,str(Path(r'C:\Users\Kinga\Desktop\MAGISTERKA\Graph-Vizualisation-Rating-Metric-3').resolve()))

from graph_generating_script import *
from graph_metric_script import *
import matplotlib.pyplot as plt
import igraph as ig
import numpy as np
import pickle

### connect all dataframes and generate coorelation for all

In [ ]:
annotations = pd.read_excel('BA_annotations_scores.xlsx')

In [ ]:
import os 
posdf_dir = 'pos_dfs'
graph_dir = 'graph_objects'


calculated_components_df = pd.DataFrame(columns=['posdf_id', 'score', 'node_distribution', 'distance_to_borderlines', 'edge_length_sum', 'edge_node_distance_contribution', 'count_edge_crossings','communities_closeness', 'sum_of_angles', 'symmetry'])

# Iterate through files in the posdf_dir
for posdf_file in os.listdir(posdf_dir):
    # reading data
    if posdf_file.endswith('.csv'):
        posdf = pd.read_csv(os.path.join(posdf_dir, posdf_file))
        print(f'Loaded {posdf_file}')

    graph_id = posdf_file.split('_')[0]
    layout = posdf_file.split('.')[0]
    layout = layout[2:]
    score = annotations[annotations['graph_id'] == int(graph_id)][layout].values[0]


    graph_file = 'graph_'+graph_id+'.pkl' 
    with open(os.path.join(graph_dir, graph_file), 'rb') as f:
        G = pickle.load(f)
    components_dict = {
        'posdf_id': posdf_file,
        'score': score,
        'node_distribution': node_distribution(posdf),
        'distance_to_borderlines': distance_to_borderlines(posdf),
        'edge_length_sum': edge_length_sum(G, posdf),
        'edge_node_distance_contribution': edge_node_distance_contribution(G, posdf)[0],
        'count_edge_crossings': count_edge_crossings(G, posdf),
        'communities_closeness' : intra_cluster_distance(G, posdf)['overall_sum'],
        'sum_of_angles' : sum_of_angles(G, posdf),
        'symmetry' : measure_graph_symmetry(G, posdf)
    }
    new_row_df = pd.DataFrame([components_dict])
    calculated_components_df = pd.concat([calculated_components_df, new_row_df], ignore_index=True)